In [2]:
import numpy as np
import pandas as pd
import os
import sys

# Add the directory containing the VotingModel to the Python path
sys.path.append(os.path.abspath('/Users/idrees/Code/govxs/'))
from util.voting_rules import mean_aggregation, median_aggregation, quadratic_aggregation
from model.VotingModel import VotingModel

def utility(voter_preferences, outcome):
    """Calculate the utility of a voter given their preferences and the outcome."""
    return -np.sum(np.abs(voter_preferences - outcome))

def evaluate_group_strategyproofness(model, coalition_size=3):
    group_strategyproofness_results = []
    for method in ["mean", "median", "quadratic"]:
        truthfully_voted_outcome = model.allocate_funds(method)
        group_strategyproof = True

        for _ in range(100):  # Number of random coalitions to test
            coalition = np.random.choice(model.num_voters, coalition_size, replace=False)
            original_utilities = [utility(model.voting_matrix[i], truthfully_voted_outcome) for i in coalition]

            # Create a strategic voting scenario
            strategic_voting_matrix = model.voting_matrix.copy()
            for i in coalition:
                strategic_voting_matrix[i] = np.random.rand(model.num_projects)

            # Update the model's voting matrix
            model.voting_matrix = strategic_voting_matrix

            # Calculate the new outcome with strategic votes
            strategically_voted_outcome = model.allocate_funds(method)
            new_utilities = [utility(model.voting_matrix[i], strategically_voted_outcome) for i in coalition]

            # Check if at least one member of the coalition is not better off
            if all(new_utilities[i] > original_utilities[i] for i in range(coalition_size)):
                group_strategyproof = False
                break

            # Reset the voting matrix to the original
            model.voting_matrix = strategic_voting_matrix

        group_strategyproofness_results.append({
            "voting_rule": method,
            "group_strategyproof": group_strategyproof
        })

    return pd.DataFrame(group_strategyproofness_results)

# Initialize the model
num_voters = 144
num_projects = 600
total_op_tokens = 30e6
model = VotingModel(num_voters=num_voters, num_projects=num_projects, total_op_tokens=total_op_tokens)
model.step()

# Evaluate group-strategyproofness
group_strategyproofness_results = evaluate_group_strategyproofness(model)
group_strategyproofness_results

,voting_rule,group_strategyproof
0,mean,False
1,median,False
2,quadratic,False
